<a href="https://colab.research.google.com/github/LorenzoTinfena/BestSpiderWeb/blob/master/BestSpiderWeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BestSpiderWeb
## Problem
A city without roads has a grain producer, an egg producer and a hotel.
The mayor also wants to build a pasta producer and a restaurant in the future. He also wants to build roads like in the picture, so that the producer can easily take the wheat and eggs to make pasta, and the restaurant can easily buy pasta, welcome hotel people, and buy eggs for other preparations.
**Goal:** to build roads costs, you have to make them as short as possible.

In a graph with fixed edges and with 2 types of nodes, one with a fixed position, the other with a variable position Objective: To find the positions of the variable nodes in order to have the sum of the length of the minor arcs